# ノード数のスケジュール設定を変更する
---

workerノード数のスケジュールを変更します。

## パラメータの指定

スケジュールを変更するのに必要となるパラメータを指定します。

### UnitGroup名

対象となるCoursewareHubのUnitGroup名を指定します。

VCノードを作成時に指定したUnitGroup名を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

UnitGroup名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

#### チェック

指定されたグループ名が妥当なものであることをチェックします。

managerノードにアクセスできることを確認します。

In [ ]:
target_group = f'{ugroup_name}_manager'
!ansible {target_group} -m ping

managerノードを管理者として操作できることを確認します。

In [ ]:
!ansible {target_group} -b -a 'whoami'

指定されたUnitGroupでスケジュール設定が行われていることを確認します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)
if gvars.get("vcnode_schedule") is None:
    raise RuntimeError("まだスケジュールの設定が行われていません")

## スケジュール定義の変更

### 状態確認

現在設定されているスケジュールの定義内容を確認します。

スケジュールの内容を表示するための環境を準備します。

In [ ]:
from pathlib import Path
!ansible-playbook -v -e venv_dir={str(Path.cwd() / ".venv")} playbooks/setup-local-venv.yml

現在設定されているスケジュールの内容を表示します。

In [ ]:
%run scripts/schedule.py
description_schedule(ugroup_name)

### スケジュールの指定

スケジュールの内容（期間、ノード数）または、各スケジュールの優先度を変更します。変更しない場合はこの節をスキップして次の節に進んでください。

ひとつのスケジュールを定義するには、期間を`begin`(開始時刻)と`end`(終了時刻)で指定し、その期間におけるworkerノード数を`node_count`で指定します。この３つの値をpythonのdictで指定します。例えば毎日22:00から翌朝7:00まではworkerノード数を1にするスケジュールは以下のような値で定義します。

```python
{
    "begin": "22:00",
    "end": "7:00",
    "node_count": 1,
}
```

期間は日毎、週毎、月毎、年毎、特定日時の指定が行えます。

日毎の場合は`22:00`のように`{時}:{分}`の形式で指定します。

週毎の場合は`Sat 0:00`のように`{曜日} {時}:{分}`の形式で指定します。曜日は`Sunday`、`日曜日`、`Sun`、`日`のような値で指定を行います。

月毎の場合は`1日 0:00`のように`{日} {時}:{分}`の形式で指定します。日は`1日`、`1`のような値で指定を行います。

年毎の場合は`12/29 0:00`のように`{月}/{日} {時}:{分}`の形式で指定します。

特定日時の場合は`2025/4/1 0:00`のように`{年}/{月}/{日} {時}:{分}`の形式で指定します。

期間の開始と終了は同じ粒度（日毎、週毎、月毎、年毎、特定日時）を指定する必要があります。

次のセルを実行すると現在設定されているスケジュール定義の`schedule_list`を表示します。

In [ ]:
%run scripts/schedule.py
show_schedule_list(ugroup_name)

次のセルでスケジュール定義のリストを指定してください。現在の設定を一部引き継ぐ場合は上のセルの出力結果から必要な箇所をコピーしてください。

複数のスケジュール期間が重複した場合、リストで先に指定したものが優先されます。

In [ ]:
schedule_list = [

]

指定したスケジュール定義の内容を確認します。次のセルの実行結果がエラーになる場合は指定されたパラメータに問題があります。上のセルで指定した`schedule_list`の値を修正してください。

In [ ]:
import sys
%run scripts/schedule.py

params = {name: value for name, value in vars().items() if name.startswith("schedule")}
params["schedule_down_type"] = schedule_down_type if "schedule_down_type" in vars() else "power_down"
params["schedule_default_nodes"] = (
    schedule_default_compute_nodes if "schedule_default_nodes" in vars() else gvars["vcnode_schedule"]["default"]["node_count"]
)
try:
    description_schedule(ugroup_name, params, "node_count")
except ValueError as e:
    print(f"ERROR: {e}", file=sys.stderr)
    raise

### 通常時のノード数

通常時のworkerノード数を変更します。変更しない場合はこの節をスキップして次の節に進んでください。

現在の通常時ノード数を確認します。

In [ ]:
print(gvars["vcnode_schedule"]["default"]["node_count"])

通常時のworkerノード数を次のセルで指定してください。

In [ ]:
# (例)
# schedule_default_nodes = 10

schedule_default_nodes = 

### ノードの停止方法

ノードの停止方法を変更します。変更しない場合はこの節をスキップして次の節に進んでください。

現在のノード停止方法を確認します。

In [ ]:
print(gvars["vcnode_schedule"]["default"]["down_type"])

次のセルで`deleted`(削除)と`power_down`(電源オフ)のどちらかを指定してください。

In [ ]:
# (例)
# schedule_down_type = "deleted"           # ノードを削除する
# schedule_down_type = "power_down"  # ノードの電源をオフする

schedule_down_type = 

### 新たなタスクの受付を停止する期間の指定

ノードを停止する前に、タスクの受付を停止し新たなコンテナを起動できないようにする期間を指定します。現在の設定値を変更しない場合はこの節をスキップして次の節に進んでください。

現在、設定している値を確認します。

In [ ]:
print(gvars["vcnode_schedule"]["default"]["drain_time"])

Availabilityを`Pause`にする時間を次のセルで指定してください。指定の単位は分となります。

In [ ]:
# (例)
# schedule_drain_time = 60

schedule_drain_time = 

### 設定内容の確認

ここまでで指定した内容を確認します。

In [ ]:
%run scripts/schedule.py
params = {
    "schedule_default_nodes": gvars["vcnode_schedule"]["default"]["node_count"],
    "schedule_down_type": gvars["vcnode_schedule"]["default"]["down_type"],
    "schedule_drain_time": gvars["vcnode_schedule"]["default"]["drain_time"],
    "schedule_list": get_schedule_list(ugroup_name),
}
params.update({name: value for name, value in vars().items() if name.startswith("schedule")})
description_schedule(ugroup_name, params)

### 保存

新たに指定したスケジュール定義をgroup_varsに保存します。

In [ ]:
update_group_vars(
    ugroup_name,
    vcnode_schedule=get_schedule_definition(ugroup_name, params),
)

## 配備

スケジュールの定義ファイルを配置します。

In [ ]:
!ansible-playbook -Dv -l {target_group} playbooks/deploy-schedule-cfg.yml

systemdタイマーの状態を確認します。タイマーによる次回の実行では新たな定義ファイルに沿ったノード構成とする処理を行います。

In [ ]:
!ansible {target_group} -b -a 'systemctl list-timers cwh-worker-node.timer'